In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

from sklearn.datasets import make_blobs

from sklearn import svm

## Главная идея

Решающее дерево предсказывает знацение целевой переменной основываясь на применении последовательности простых решаюших правил - уловий на основе которых производится деление данных на две части - удовлетворяющих и неудовлетворяющих условию.

Хотя обобщающая способность решающих деревьев невысока, их предсказания вычисляются довольно просто, из-за чего решающие деревья часто используют как основа для построения ансамблей — моделей, делающих предсказания на основе агрегации предсказаний других моделей, например случайный лес и градиентный бустинг на решаюшем дереве.

## Простой пример решающего дерева

Начнём с небольшого примера. На картинке ниже изображено дерево, построенное для задачи классификации на пять классов:

<img src="Pict/tree_basic.png" width="300"/>

Объекты в этом примере имеют два признака с вещественными значениями: $X$ и $Y$. Решение о том, к какому классу будет отнесён текущий объект выборки, будет приниматься с помощью прохода от корня дерева к некоторому листу.

В каждом узле этого дерева находится условие. Если условие верено для текущего примера из выборки, мы переходим в правую ветвь, если нет — в левую. В данном примере все условия — это просто взятие порога по значению какого-то признака:

$B(x,j,t)=[x_j\leq t]$

В листьях записаны предсказания (например, метки классов). Как только мы дошли до листа, мы присваиваем объекту ответ, записанный в вершине.

На картинке ниже визуализирован процесс построения решающих поверхностей, порождаемых деревом (правая часть картинки):

<img src="Pict/tree_basic_2.png" width="600"/>

**Внимание!** В ML принято проверять условие $\leq$ и истинный результат отправлять в левую часть, а ложный в правую. Схема в общем верная :)

Каждый условие порождает разделение текущего подмножества пространства признаков на две части. На первом этапе, когда происходило деление по $[X\leq x_1]$, вся плоскость была поделена на две соответствующие части. На следующем уровне часть плоскости, для которой выполняется $[X\leq x_1]$, была поделена на две части по значению второго признака $[Y\leq y_1]$ — так образовались области 1 и 2. То же самое повторяется для правой части дерева — и так далее до листовых вершин: получится пять областей на плоскости. 

Теперь любому объекту выборки будет присваиваться один из пяти классов в зависимости от того, в какую из образовавшихся областей он попадает.

## Определение решающего дерева

Пусть задано бинарное дерево, в котором:
- каждой внутренней вершине $\nu$ приписан предикат(условие): $B_\nu: X \to \{0,1\}$ или $\{True, False\}$ ;
- каждой листовой вершине $\nu$ приписан прогноз $c_\nu \in Y$, где $Y$ — область значений целевой переменной (в случае классификации листу может быть также приписан вектор вероятностей классов).

В ходе предсказания осуществляется проход по этому дереву к некоторому листу. Для каждого объекта выборки $x$ движение начинается из корня. В очередной внутренней вершине $\nu$ проход продолжится вправо, если $B_\nu(x)=1$, и влево, если $B_\nu(x)=0$. Проход продолжается до момента, пока не будет достигнут некоторый лист, и ответом алгоритма на объекте $x$ считается прогноз $c_\nu$, приписанный этому листу.

Предикат $B_\nu(x,j,t)$  может иметь, вообще говоря, произвольную структуру, но, как правило, на практике используют просто сравнение с порогом по какому-то $j$-му признаку:

$$B_\nu(x,j,t) = [x_j\leq t]$$


При проходе через узел дерева с данным предикатом объекты будут отправлены в правую ветвь, если значение $j$-го признака у них меньше либо равно $t$, и в левое — если больше.

Из структуры дерева решений следует несколько интересных свойств:
- выученная функция является кусочно-постоянной, из-за чего производная равна нулю везде, где задана. Следовательно, о градиентных методах при поиске оптимального решения можно забыть;
- дерево решений (в отличие от, например, линейной модели) не сможет экстраполировать зависимости за границы области значений обучающей выборки (обычно, без дополнительных фич);
- дерево решений способно идеально приблизить обучающую выборку и ничего не выучить (то есть такой классификатор будет обладать низкой обобщающей способностью): для этого достаточно построить такое дерево, в каждый лист которого будет попадать только один объект. Следовательно, при обучении нам надо не просто приближать обучающую выборку как можно лучше, но и стремиться оставлять дерево как можно более простым, чтобы результат обладал хорошей обобщающей способностью.

Вычислительная сложножность опримального алгоритма $O(N^2D)$, где $N$ - количество строк в данных,  $D$ - количество признаков на один шаг алгоритма. Это достаточно много и, и кстати, не является NP-полной задачей, поэтому строят жадные алгоритмы.

## Жадный алгоритм (локально оптимальных решений) построения решающего дерева

Пусть $X$ — исходное множество объектов обучающей выборки, а $X_m$ — множество объектов, попавших в текущий лист (в самом начале $X=X_m$). Тогда жадный алгоритм можно верхнеуровнево описать следующим образом:
1. Создаём вершину $\nu$.
2. Если выполнен критерий остановки $\text{Stop}(X_m)$, то останавливаемся, объявляем эту вершину листом и ставим ей в соответствие ответ $\text{Ans}(X_m)$, после чего возвращаем её.
3. Иначе: находим разбиение (иногда ещё говорят сплит) $B_{j,t}$, который определит наилучшее разбиение текущего множества объектов на две подвыборки $X_l$ и $X_r$, минимизируя функцию ошибки $L(X_m, j ,t)$ разбиения $[x_j\leq t]$.
Для  и  рекурсивно повторим процедуру.
4. Для  $X_l$ и $X_r$  рекурсивно повторим процедуру.


Данный алгоритм содержит в себе несколько вспомогательных функций, которые надо выбрать так, чтобы итоговое дерево было способно минимизировать функцию потерь:

1. $\text{Ans}(X_m)$, вычисляющая ответ для листа по попавшим в него объектам из обучающей выборки, может быть, например:
    - в случае задачи классификации — меткой самого частого класса или оценкой дискретного распределения вероятностей классов для объектов, попавших в этот лист;
    - в случае задачи регрессии — средним, медианой или другой статистикой;
    - простой моделью, например, листы в дереве, решающем задачу регрессию, могут быть линейными функциями или синусоидами, обученными на данных, попавших в лист. Впрочем, везде ниже мы будем предполагать, что в каждом листе просто предсказывается константа.
2. Критерий остановки  — функция, которая решает, нужно ли продолжать ветвление или пора остановиться.
Это может быть какое-то тривиальное правило: например, остановиться только в тот момент, когда объекты в листе получились достаточно однородными и/или их не слишком много. Более детально мы поговорим о критериях остановки в параграфе про регуляризацию деревьев.

3. Критерий ветвления  — пожалуй, самая интересная компонента алгоритма. Это функция, измеряющая, насколько хорош предлагаемое разделение (сплит). Чаще всего эта функция оценивает, насколько улучшится некоторая финальная метрика качества дерева в случае, если получившиеся два листа будут конечным результатом, по сравнению с ситуацией, когда сама исходная вершина является листом. Выбирается такой сплит, который даёт наиболее существенное улучшение. Впрочем, есть и другие подходы.

При этом строгой теории, которая бы связывала оптимальность выбора разных вариантов этих функций и разных метрик классификации и регрессии, в общем случае не существует. Однако есть набор интуитивных и хорошо себя зарекомендовавших соображений.

## Критерии ветвления (разбиения) 

Пусть в вершину  $\nu$ оказалась выборка $X_m$.

Пусть $L(X_m, j, t)$ функция ошибки условия разбиения $[x_j\leq t]$.

Ищем наилучшие параметры $j$ и $t$ перебором:
$$ L(X_m, j, t)\to \min_{j,t}$$

Разбиваем $X_m$ на две части:
$$ X_l = \{x\in X_m\; | \; [x_j\leq t] \}$$
$$ X_r = \{x\in X_m\; | \; [x_j > t] \}$$

Тогда наша функция ошибки будет иметь вид:
$$L(X_m, j, t) = \frac{|X_l|}{|X_m|}H(X_l) + \frac{|X_r|}{|X_m|}H(X_r)$$
где $\frac{|X_l|}{|X_m|}$ - доля ответов, $H(X_l)$ - функция характерезующая разброс ответов от 0 до 1, при минимальном разбросе $H(X)=0$, при максимальном - 1.

Принято называть $H(X)$ - критерием информативности, чем он ниже тем лучше объекты в листе можно приблизить константным значением.

### Критерий информативности в задачах классификации

1. Доля обьектов класса $k$ в выборке $X$: 
$$p_k = \frac{1}{|X|}\sum_{i\in X}[y_i=k],$$
а наиболее оптимальным предсказанием  в вершины/листа будет наиболее частотный класс ($k$), тогда выражение для информативности упростится следующим образом:
$$ H(X_m) = \frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} [y_i \ne k] = 1 - p_{k}$$

2. Критерий Джини (важно не путать с коэффициентом Джинни)
$$ H(X_m) = \sum_{k = 1}^K p_k (1 - p_k)$$
Если $p_1=0,\;p_2=...=p_k=0$, то $H(x)=0$.<br>
Критерий Джини допускает и следующую интерпретацию: $H(X_m)$ равно математическому ожиданию числа неправильно классифицированных объектов в случае, если мы будем приписывать им случайные метки из дискретного распределения, заданного вероятностями $(p_1,...,p_k)$.<br>
Иначе говоря - вероятность ошибки классификатора, который выдает ответы пропорционально $p_k$.

3. Критерий энтропии:
$$H(X_m) = -\sum_{k = 1}^K p_k \log p_k$$
    при этом считаем $0\log 0 = 0$. <br>
Если $p_1=0,\;p_2=...=p_k=0$, то $H(x)=0$.<br>

Информационная энтропия Шеннона измеряет непредсказуемость реализации случайной величины. <br>
В оригинальном определении, правда, речь шла не о значениях случайной величины, а о символах (первичного) алфавита, так как Шеннон придумал эту величину, занимаясь вопросами кодирования строк. Для данной задачи энтропия имеет вполне практический смысл — среднее количество битов, которое необходимо для кодирования одного символа сообщения при заданной частоте символов алфавита.

Так как $p_k\in[0,1]$, энтропия неотрицательна. Если случайная величина принимает только одно значение, то она абсолютно предсказуема и её энтропия равна $-1\log(1)=0$

Наибольшего значения энтропия достигает для равномерно распределённой случайной величины — и это отражает тот факт, что среди всех величин с данной областью значений она наиболее «непредсказуема». Для равномерно распределённой на множестве $\{1,...,K\}$ случайной величины значение энтропии будет равно:

$$-\sum_{k = 1}^K \frac{1}{K}\log\frac{1}{K} = \log K$$

Фактически критерий энтропии это мера отличия распределения классов от вырожденного, т.е. если распредеоение вырождено, то энтрория равна 0, так как в этом распределении нет ничего неожиданного, мы всегда знаем ответ, если же распределение классов равномерно, то этропия максимальна, так как уровень неожиданности максимально возможный.

### Неоптимальность полученных критериев

Казалось бы, мы вывели критерии информативности для всех популярных задач, и они довольно логично следуют из их постановок, но получилось ли у нас обмануть NP-полноту и научиться строить оптимальные деревья легко и быстро?

Конечно, нет. Простейший пример — решение задачи XOR с помощью жадного алгоритма и любого критерия, который мы построили выше:
<img src="Pict/tree_xor.png" width="600"/>

Вне зависимости от того, что вы оптимизируете, жадный алгоритм не даст оптимального решения задачи XOR. Но этим примером проблемы не исчерпываются. Скажем, бывают ситуации, когда оптимальное с точки зрения выбранной метрики дерево вы получите с критерием ветвления, построенным по другой метрике (об этом чуть позже).

**Примечание:** задача XOR(исключающее ИЛИ) это задача с образами по углам квадрата (0,0) и (1,1) это класс 0, (0,1) и (1,0) это класс 1. Так принято ее называть в литературе по ML и на собеседованиях. Это прямая аналогия с результатом побитной операции XOR (0^0=1, 0^1=0, 1^0=0, 1^1=1).

### Критерий информативности в задачах регрессии

1. MSE. Достаточно очевидно, что оптимальным предсказанием для задачи минимизации MSE является нахождение среднего значения. Тогда
$$ H(X_m) = \sum\limits_{(x_i, y_i) \in X_m}\frac{\left(y_i - \overline{y} \right)^2}{|X_m|}, ~ \text{где} ~ \overline{y} = \frac{1}{\vert X_m \vert} \sum_i y_i$$

То есть при жадной минимизации MSE, информативность — это оценка дисперсии таргетов для объектов, попавших в лист. 

Получается очень стройная картинка: оценка значения в каждом листе — это среднее, а выбирать сплиты надо так, чтобы сумма дисперсий в листьях была как можно меньше.

2. MAE. Случай средней абсолютной ошибки так же прост: в листе надо предсказывать медиану, ведь именно медиана таргетов для обучающих примеров минимизирует MAE нашего предсказателя.

В качестве информативности выступает абсолютное отклонение от медианы:
$$H(X_m) = \sum\limits_{(x_i, y_i) \in X_m}\frac{|y_i -  \text{median}(Y)|}{|X_m|}$$

## Критерии останова и стрижки

1. В узле не долно быть менее 5 объектов. Если в узле менее 5 объектов он становится листом, и дальше не дробится `min_samples_split`

2. Ограничивать в глубину - жестко через `max_depth`, мягко через `min_samples_leaf` - не даст расти в глубину если конечные листы будут содержать менее заданного значения.

Такие приемы позволяют стричь дерево, ограничивая и направляя его рост.

**НО ЭТО ЛИШНЕЕ!!! ОДИНОЧНОЕ ДЕРЕВО ПРАКТИЧЕСКИ НЕВОЗМОЖНО ВСТРЕТИТЬ, ВСЕ РЕШАЮТ АНСАМБЛИ.**